In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,135 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Get:13 http://security.ubuntu.com

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"




In [11]:
import urllib.request

# URL of the CSV file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file
urllib.request.urlretrieve(url, "/tmp/home_sales.csv")

# Read the downloaded file into a DataFrame
df = spark.read.csv("file:///tmp/home_sales.csv", header=True, inferSchema=True)


In [14]:
# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


In [15]:
# Display the schema of the DataFrame
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [19]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import round

# Calculate the average price for a four-bedroom house sold per year
average_price_per_year = df.filter(df['bedrooms'] == 4)\
    .groupBy('date_built')\
    .avg('price')\
    .select('date_built', round('avg(price)', 2).alias('avg_price'))\
    .orderBy('date_built')

# Show the result
average_price_per_year.show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [20]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

from pyspark.sql.functions import round

# Calculate the average price for homes with 3 bedrooms and 3 bathrooms, per year
average_price_per_year = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))\
    .groupBy('date_built')\
    .avg('price')\
    .select('date_built', round('avg(price)', 2).alias('avg_price'))\
    .orderBy('date_built')

# Show the result
average_price_per_year.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [21]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

from pyspark.sql.functions import round


average_price_per_year = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] >= 2000))\
    .groupBy('date_built')\
    .avg('price')\
    .select('date_built', round('avg(price)', 2).alias('avg_price'))\
    .orderBy('date_built')

# Show the result
average_price_per_year.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [27]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
from pyspark.sql.functions import avg, round

# Calculate the average price per "view" rating for homes with an average price greater than or equal to $350,000
average_price_per_view = df.groupBy('view')\
    .agg(round(avg('price'), 2).alias('avg_price'))\
    .filter('avg_price >= 350000')\
    .orderBy('view', ascending=False)

# Show the result
average_price_per_view.show()

# Calculate and print the run time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.0001270771026611328 seconds ---


In [25]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")


In [26]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [29]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.




# Calculate the average price per "view" rating for homes with an average price greater than or equal to $350,000 using cached data
start_time_cached = time.time()
average_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")
cached_runtime = time.time() - start_time_cached

# Show the result and runtime comparison
print("Cached Runtime: %.9f seconds" % cached_runtime)
average_price_per_view_cached.show()

# Calculate the runtime for the uncached query
start_time_uncached = time.time()
average_price_per_view_uncached = df.groupBy('view')\
    .agg(round(avg('price'), 2).alias('avg_price'))\
    .filter('avg_price >= 350000')\
    .orderBy('view', ascending=False)
uncached_runtime = time.time() - start_time_uncached

# Show the result and runtime comparison
print("Uncached Runtime: %.9f seconds" % uncached_runtime)
average_price_per_view_uncached.show()


Cached Runtime: 0.034354448 seconds
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Uncached Runtime: 0.081413507 seconds
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows



In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Write the DataFrame to parquet partitioned by the "date_built" field
# Write the DataFrame to parquet partitioned by the "date_built" field
df.write.partitionBy("date_built").parquet("/content/partitioned_parquet_file")

# Create a temporary table for the parquet data
spark.read.parquet("/content/partitioned_parquet_file").createOrReplaceTempView("home_sales_partitioned")



In [33]:
# 11. Read the parquet formatted data.
# Read the partitioned parquet file into a DataFrame
df_partitioned = spark.read.parquet("/content/partitioned_parquet_file")




In [34]:
# 12. Create a temporary table for the parquet data.
# Create a temporary table for the parquet data
df_partitioned.createOrReplaceTempView("home_sales_partitioned")

# Show the DataFrame
df_partitioned.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [35]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


start_time_parquet = time.time()

# Calculate the average price per "view" rating for homes with an average price greater than or equal to $350,000 using parquet data
average_price_per_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_partitioned
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

# Show the result and calculate the runtime
parquet_runtime = time.time() - start_time_parquet
print("Parquet Runtime: %.9f seconds" % parquet_runtime)
average_price_per_view_parquet.show()

# Compare with the cached runtime
print("Cached Runtime: %.9f seconds" % cached_runtime)





Parquet Runtime: 0.034370422 seconds
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Cached Runtime: 0.034354448 seconds


In [36]:
# 14. Uncache the home_sales temporary table
spark.catalog.uncacheTable("home_sales")


In [37]:
# 15. Check if the home_sales is no longer cached

# 15. Check if the home_sales table is no longer cached
is_cached = spark.catalog.isCached("home_sales")
if not is_cached:
    print("home_sales table is not cached.")
else:
    print("home_sales table is still cached.")


home_sales table is not cached.
